In [ ]:
#  import main packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

In [ ]:
# define variables
immo_vis_dir = "../../data/immo_vis/"
parquet_extension = ".parquet"
metropole_appartement_file = "ventes-metropole-appartement" + parquet_extension
metropole_maison_file = "ventes-metropole-maison" + parquet_extension

In [ ]:
# import appartement file

start_path = Path(immo_vis_dir)
final_path = start_path / metropole_appartement_file
final_path.as_posix
df= pd.read_parquet(final_path.as_posix())

In [ ]:
#  print main infos on appartmeent file

nb_rows= df.shape[0]
nb_cols= df.shape[1]
print ( f" {metropole_appartement_file} rows {nb_rows} columns {nb_cols}")
print (df.shape)
print (df.columns)
print (df.info())
print (df.describe())



In [ ]:
#  print columns modalities
for i in df.columns:
    print(f'modalities  {i} are : ',(df[i].nunique()))

In [ ]:
#  print comlmns nan values
for i in df.columns:
    print(f' {i} nan values {df[i].isna().sum()}  {(100*df[i].isna().sum()/len(df)):.2f}%')

In [ ]:
# analyze type de bien
print (df['typedebien'].value_counts()) 
print (df['logement_neuf'].value_counts()) 
# print (df['typedebien'].value_counts(normalize=True)*100)
appartement_anciens = df.loc[df["typedebien"] == "a"]
appartement_neufs = df.loc[df["typedebien"] == "an"]

logement_anciens = df.loc[df["typedebien"] == "o"]
logement_neufs = df.loc[df["typedebien"] == "n"]

print ("appartement_neufs", appartement_neufs["logement_neuf"].value_counts())
print ("appartement_anciens", appartement_anciens["logement_neuf"].value_counts())


# select apppr
print ("appartements anciens , logement ancien ", appartement_anciens[appartement_anciens["logement_neuf"]== "o"].shape[0])
print ("appartements anciens , logement neuf ", appartement_anciens[appartement_anciens["logement_neuf"]== "n"].shape[0])

print ("appartements neufs , logement neuf ", appartement_neufs[appartement_neufs["logement_neuf"]== "n"].shape[0])
print ("appartements neufs , logement ancien ", appartement_neufs[appartement_neufs["logement_neuf"]== "o"].shape[0])

print (f" prix vente m2 appartements anciens  {appartement_anciens['prix_m2_vente'].median()}" )
print (f" prix vente m2 appartement_neufs {appartement_neufs['prix_m2_vente'].median()}")

print (f" prix vente m2 logements anciens  {logement_anciens['prix_m2_vente'].median()}" )
print (f" prix vente m2 logements neufs {logement_neufs['prix_m2_vente'].median()}")

print (f" dpeL appartements anciens  {appartement_anciens['dpeL'].mode()[0]}" )
print (f" dpeL  appartement_neufs {appartement_neufs['dpeL'].mode()[0]}")

print ("logement_neuf is not a relaible criteria")

In [ ]:
#  modify logement_neuf according to type de bien
mask = (df["typedebien"] == "an") & (df["logement_neuf"]== "o")
df.loc[mask,'logement_neuf'] ="n"

mask = (df["typedebien"] == "a") & (df["logement_neuf"]== "n")
df.loc[mask,'logement_neuf'] ="o"

# replace an by a
df.loc[:,'typedebien'] ="a"

# process logement neuf as integer
df['logement_neuf'] = df['logement_neuf'].replace(['o','n'],[1,0])

print (df.typedebien.value_counts())
print (df.logement_neuf.value_counts())



## Eliminate columns that are not relevant

In [ ]:
#rendre index=idannonce pour retrouver la ligne initiale
df.set_index('idannonce',drop=True,inplace=True)

#supprimer les colonnes non pertinentes pour les maisons
df.drop(columns=['type_annonceur', 'typedebien', 'typedetransaction',
       'prix_maison', 'prix_terrain', 'mensualiteFinance',"surface_terrain",
    #    'dpeC', #pas d'information supplémentaire sur dpeC par rapport à dpeL
    #    'date','eau',
       'categorie_annonceur',  'date', 'INSEE_COM', 'IRIS', 'duree_int',
       'TYP_IRIS_x', 'TYP_IRIS_y', 'GRD_QUART', 'UU2010', 'REG', 
       'loyer_m2_median_n6', 'nb_log_n6', 'taux_rendement_n6',
       'loyer_m2_median_n7', 'nb_log_n7', 'taux_rendement_n7'], 
       inplace=True)

## Process prix_m2_vente

In [ ]:
# pre-analyse prix de vente au m2

plt.figure(figsize=(22,6))
plt.subplot(151)
sns.boxplot(df['prix_m2_vente'])
plt.subplot(152)
sns.histplot(df['prix_m2_vente'])
plt.subplot(153)
sns.scatterplot (df,x="surface",y="prix_m2_vente")
plt.show()


In [ ]:


# remove outliers

colonnes_num_prim = ['surface',  'prix_m2_vente']

# Création du masque en utilisant apply
mask = df[colonnes_num_prim].apply(lambda x: (x >= x.quantile(0.005)) & (x <= x.quantile(0.995))| x.isna()).all(axis=1)
#ne garder que les lignes qui sont entre q=0.005 et q=0.995
df=df[mask]
#Vérification des résultats
df[colonnes_num_prim].describe()

(df['prix_m2_vente']*df['surface']-df['prix_bien']).describe()


plt.figure(figsize=(22,6))
plt.subplot(151)
sns.boxplot(df['prix_m2_vente'])
plt.subplot(152)
sns.histplot(df['prix_m2_vente'])
plt.subplot(153)
sns.scatterplot (df,x="surface",y="prix_m2_vente")
plt.show()

# 

## Traitement des colonnes  

In [ ]:
# analyse NA values
import matplotlib.ticker as mtick

missing_values = df.isna().sum()
missing_values = missing_values[missing_values > 0].sort_values(ascending=False)
# full_columns = df.columns[ df.isna().any() == False ]
print (f" nb_rows {nb_rows} nb_cols {nb_cols}")
# print(f"columns without Nan values {full_columns.size} / {nb_cols}" )
# print (f"columns without Nans {full_columns}")
# print (missing_values)
plt.figure(figsize=(20,6))
ax = plt.subplot(121)
plt.plot (missing_values.index,missing_values.values)
plt.axhline(y=int(nb_rows/2), color='r', linestyle='--', label='50%')
plt.axhline(y=int(nb_rows*0.9), color='b', linestyle='--', label='90%')
plt.legend()

plt.xticks(rotation=80)
plt.title(f"Missing values  nb-rows = {nb_rows}")
ax2 = plt.subplot(122)
missing_values_percent = (missing_values/nb_rows)*100.0
plt.plot (missing_values_percent.index,missing_values_percent.values)
plt.axhline(y=50.0, color='r', linestyle='--', label='50%')
plt.axhline(y=90.0, color='b', linestyle='--', label='90%')
plt.xticks(rotation=80)
ax2.yaxis.set_major_formatter(mtick.PercentFormatter(100)) 
plt.title(f"Missing values  percentage")
plt.legend()
plt.show()


## Process columns with too many NANs

In [ ]:
# detect  columns with too many NANs (90%)
print (df.shape)
threshold = .9
nan_cols = []
for name,_ in df.items() :
    if df[name].isna().sum()/nb_rows> threshold :
        nan_cols.append(name) 
print (f"empty cols {nan_cols}")

In [ ]:
# remove nan cols > threshold
df = df.drop(columns=nan_cols)

## Process numerical columns

In [ ]:
# scatter plot of numerical variables / prix_m2_vente 
columns = df.select_dtypes(include='number').columns.tolist()
df.sort_values(by="prix_m2_vente",ascending=True)
nrows = int(len(columns)/4) +1
fig , axes = plt.subplots(nrows = nrows, ncols = 4, figsize = (30,nrows*4))
index = 0
for column in columns :
    if column not in ["idannonce","date"] :
        sns.scatterplot(ax=axes[int(index/4),index%4],y=df["prix_m2_vente"],x=df[column])
        index = index +1


In [ ]:
# remove outliers .99 % except nb_logements_copro, charges_copro

#  etage contains negative values
df.loc[:,'etage'] = df['etage'].abs() 

columns = ['bain', 'nb_toilettes', 'nb_pieces','eau','bain','nb_pieces','nb_terraces','balcon','places_parking','nb_etages','etage']
#limiter les colonnes à q=0.99 pour éliminer les outliers
threshold = 0.99
for column in columns :
    df.loc[:,column] = df[column].clip(upper=df[column].quantile(threshold))


In [ ]:
# display 
nrows = int(len(columns)/4) +1
fig , axes = plt.subplots(nrows = nrows, ncols = 4, figsize = (30,15*nrows))
index = 0
for column in columns :
    sns.scatterplot(ax=axes[int(index/4),index%4],y=df["prix_m2_vente"],x=df[column])
    index = index +1


In [ ]:
#  remove Nan values for numerical columns
columns = df.select_dtypes(include='number').columns.tolist()
print (df[columns].isna().sum())


In [ ]:

#  place de parking
df["places_parking"] = df["places_parking"].fillna(0) 
# nombres de toilettes ( can not be guessed)
df["nb_toilettes"] = df["nb_toilettes"].fillna(1).astype(int)
# logement neuf
df["logement_neuf"] = df["logement_neuf"].fillna(0)
#  nombre de terraces
df["nb_terraces"] = df["nb_terraces"].fillna(0)
df.loc[:,"nb_terraces"] = 0

print (df[columns].isna().sum())



In [ ]:
# nb etages
print (df.nb_etages.value_counts(dropna=False))
fig = plt.figure(figsize=(16,8))
sns.countplot(data=df,x=df.nb_etages)
df["nb_etages"]= df["nb_etages"].fillna(df["nb_etages"].median())

In [ ]:
# annee de construction
print (df.nb_logements_copro.value_counts(dropna=False,normalize=True))
sns.scatterplot(y=df["prix_m2_vente"],x=df["annee_construction"])
df["annee_construction"]= df["annee_construction"].fillna(0)


In [ ]:
# nb logmeent copro
threshold=0.95
column="nb_logements_copro"
print (df[column].value_counts(dropna=False,normalize=True))
sns.scatterplot(y=df["prix_m2_vente"],x=df[column])
plt.show()
df.loc[:,column] = df[column].clip(upper=df[column].quantile(threshold))
sns.scatterplot(y=df["prix_m2_vente"],x=df[column])
plt.show()
print (df[column].median())
df[column]= df[column].fillna(0)



In [ ]:
# charges copro
threshold=0.95
column="charges_copro"
print (df[column].value_counts(dropna=False,normalize=True))
sns.scatterplot(y=df["prix_m2_vente"],x=df[column])
plt.show()
df.loc[:,column] = df[column].clip(upper=df[column].quantile(threshold))
sns.scatterplot(y=df["prix_m2_vente"],x=df[column])
plt.show()
print (df[column].median())
df[column]= df[column].fillna(0)



In [ ]:
#  remove Nan values for numerical columns
columns = df.select_dtypes(include='number').columns.tolist()
print (df[columns].isna().sum())

In [ ]:
column="dpeC"
threshold=.99
print (df[column].value_counts(dropna=False,normalize=True))
df.loc[:,column] = df[column].clip(upper=df[column].quantile(threshold))
fig = plt.figure(figsize=(16,8))
sns.scatterplot(y=df[column],x=df["dpeL"])
plt.show()
result = df.groupby("dpeL")[column].mean()
print (result)
fig = plt.figure(figsize=(16,8))
sns.scatterplot(y=result.values,x=result.index)
plt.show()

In [ ]:
#  drop dpeC : colinearity with dpeL
column="dpeC"
df = df.drop(columns=[column])

In [ ]:
#  remove Nan values for numerical columns
columns = df.select_dtypes(include='number').columns.tolist()
print (df[columns].isna().sum())


## Process categorial columns